<a href="https://colab.research.google.com/github/Howl06/practice/blob/main/imdb%20NLP%20%E6%83%85%E7%B7%92%E5%88%A4%E6%96%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84125825/84125825 [==============================] - 2s 0us/step


In [2]:

import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    # postive all data
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    # negative all data
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df
train_df = getdata("train")
test_df = getdata("test")

In [3]:
test_df

,content,sentiment
0,ROCK STAR / (2001) *** (out of four)<br /><br ...,1
1,The Cure is a fantastic film about a boy with ...,1
2,"Not having seen this film in quite some time, ...",1
3,One of the flat-out drollest movies of all-tim...,1
4,"NVA combines eastalgia-humor, military comedy ...",1
...,...,...
24995,Strangers with candy overacts in all the wrong...,0
24996,"What a disaster! Normally, when one critiques ...",0
24997,The Robot vs. the Aztec Mummy was one of the s...,0
24998,I'm not prone to ranting and my expectations w...,0


In [4]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [7]:
glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")

['/root/.keras/datasets/aclImdb/train/pos/11812_10.txt']

In [17]:
with open("/root/.keras/datasets/aclImdb/train/pos/11812_10.txt", encoding="utf-8") as f:
  print(f.read())

This movies is the best movie to watch for comic book feel. The sets, costumes and the color are just so vivid it is just like stepping into a comic book. This is the movie I think of when the Mob is mentioned, the suits, the hats and the attitudes.<br /><br />Hoffman gives comic relief as Mumbles and you can't help but feel sorry for Madonna as she tries, and fails, to win Tracy over. This movie contains all the classic mob clichés - burying people in concrete, blowing up peoples cars, tieing up the good guy and attempting to blow up his girlfriends house.<br /><br />This movie is a classic in ever sense of the word, even camera angels cry out comic book. Its so great to be able to go back to an older movie and see that someone knew how a comic should be made into a movie after seeing such mistakes as Spawn and the Hulk. <br /><br />!!!YOU HAVE TO SEE THIS MOVIE!!!!


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D, Embedding
# 3000種常用詞彙 + 1 padding(0): 美一篇文章, 每一個詞化做100維度的語意向量
layers = [
    # 沒有激活, 3001(種詞彙) * 100 -> 300100 mask_zero是否把 0 看作特殊的 "padding" 值。
    # 512 最多一篇文章只取512個常用詞彙
    Embedding(input_dim=3001, output_dim=100, mask_zero=True, input_length=512),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 512, 100)          300100    
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 300,302
Trainable params: 300,302
Non-trainable params: 0
_________________________________________________________________


In [19]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       metrics=["accuracy"],
       optimizer="adam")

In [73]:
# Tokenize: 詞彙換成數字, 建立一個3000常用詞彙辭典
from tensorflow.keras.preprocessing.text import Tokenizer
# 保留問號 驚嘆號
tok = Tokenizer(num_words=3000, filters='"?!#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n')
tok.fit_on_texts(train_df["content"])

In [74]:
# tok.word_index
# tok.index_word
# 檢查: 這個case, 標點和換行是可以去掉的
# tok.word_index["?"]
# 停用詞(忽略一些無意義的): 不用, 根據答案就會把無意義的東西調整出來

In [64]:
# 加入問號考慮
tok.word_index["?"]

666

In [75]:
import pandas as pd
# Sequenize: 把我的字轉換成數字(利用剛剛列表)
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,...,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,389,1357,7,7,51,10,605,30,1.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,57,148,11,17,6,176,151,54,548.0,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,1056,3,224,4,11,20,1067,695.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,1816,120,16,3,324,1918,705,18.0,204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,758,4,1059,309,392,1534,294,3.0,2735.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,2,419,38,93,140,11,1976,706,60.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,45,10,97,199,9,3,1454,471,650.0,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,100,109,2177,31,440,236,14,3,8.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,11,17,6,32,5,19,228,10,188.0,262.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,...,5,1,372,1706,2,190,1,233,1370,341
1,0,0,0,0,0,0,0,0,0,0,...,37,543,108,405,34,25,293,9,487,27
2,0,0,0,0,0,0,0,0,0,0,...,2,96,487,178,5,398,9,742,772,2095
3,0,0,0,0,0,0,0,0,0,0,...,48,56,6,2246,4,900,76,142,5,12
4,0,0,0,0,0,0,0,0,0,0,...,24,2547,1,1605,624,124,1,2863,49,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,32,4,1199,250,8,1,223,297,238,36
24996,0,0,0,0,0,0,0,0,0,0,...,21,167,5,78,70,452,141,27,443,302
24997,0,0,0,0,0,0,0,0,0,0,...,344,4,805,119,94,69,1822,287,690,979
24998,0,0,0,0,0,0,0,0,0,0,...,43,47,12,188,76,2,33,120,11,592


In [77]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [78]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
c = [
    ModelCheckpoint("imdb.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad,
     y_train,
     batch_size=100,
     epochs=40,
     validation_split=0.1,
     callbacks=c)

Epoch 1/40
225/225 [==============================] - 6s 27ms/step - loss: 0.5102 - accuracy: 0.7670 - val_loss: 0.5030 - val_accuracy: 0.7656
Epoch 2/40
225/225 [==============================] - 7s 32ms/step - loss: 0.3865 - accuracy: 0.8292 - val_loss: 0.4176 - val_accuracy: 0.8120
Epoch 3/40
225/225 [==============================] - 7s 29ms/step - loss: 0.3422 - accuracy: 0.8546 - val_loss: 0.4465 - val_accuracy: 0.7988
Epoch 4/40
225/225 [==============================] - 7s 30ms/step - loss: 0.3133 - accuracy: 0.8708 - val_loss: 0.4175 - val_accuracy: 0.8176
Epoch 5/40
225/225 [==============================] - 6s 29ms/step - loss: 0.2916 - accuracy: 0.8819 - val_loss: 0.3813 - val_accuracy: 0.8352
Epoch 6/40
225/225 [==============================] - 7s 30ms/step - loss: 0.2743 - accuracy: 0.8895 - val_loss: 0.4004 - val_accuracy: 0.8300
Epoch 7/40
225/225 [==============================] - 6s 29ms/step - loss: 0.2603 - accuracy: 0.8964 - val_loss: 0.3408 - val_accuracy: 0.8620

In [54]:
model.evaluate(x_test_pad, y_test) # ? !

782/782 [==============================] - 4s 5ms/step - loss: 0.3195 - accuracy: 0.8706


[0.31947749853134155, 0.870639979839325]

In [61]:
model.evaluate(x_test_pad, y_test) #　！

782/782 [==============================] - 4s 5ms/step - loss: 0.3250 - accuracy: 0.8674


[0.3249537944793701, 0.8674399852752686]

In [72]:
model.evaluate(x_test_pad, y_test) #　?

782/782 [==============================] - 4s 5ms/step - loss: 0.3132 - accuracy: 0.8715


[0.3131521940231323, 0.8714799880981445]

In [80]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.3207 - accuracy: 0.8669


[0.32074227929115295, 0.8668799996376038]